In [ ]:
!unzip Fotos.zip

In [ ]:
carpeta_imagenes = "./Gas-Meter-Counter/JPEGImages"
carpeta_anotaciones = "./Gas-Meter-Counter/Annotations"
carpeta_anotaciones_digits = "./Gas-Meter-Counter/Annotations_digits"
carpeta_salida = "./Salida recortes"
carpeta_salida_entrenar = "./Salida_para_entrenar"

#crop images
import numpy as np # linear algebra
import xml.etree.ElementTree as ET # for parsing XML
from PIL import Image # to read images
import os
import glob
import cv2 as cv
from matplotlib import pyplot as plt
import uuid

root_images=carpeta_imagenes
root_annots=carpeta_anotaciones
root_annots_digits=carpeta_anotaciones_digits

all_images=os.listdir(carpeta_imagenes)
print(f"Total images : {len(all_images)}")

breeds = glob.glob(carpeta_anotaciones)
annotation=[]
for b in breeds:
    annotation+=glob.glob(b+"/*")
print(f"Total annotation : {len(annotation)}")

breeds = glob.glob(carpeta_anotaciones_digits)
annotation_digits=[]
for b in breeds:
    annotation_digits+=glob.glob(b+"/*")
print(f"Total annotation_digits : {len(annotation_digits)}")

breed_map={}
for annot in annotation_digits:
    breed=annot.split("/")[-2]
    index=breed.split("-")[0]
    breed_map.setdefault(index,breed)



def get_digits(image):
    bpath=root_annots_digits+"/"+str(image.split(".")[0]+".xml")
    try:
      tree = ET.parse(bpath)
    except:
      return []
    root = tree.getroot()
    objects = root.findall('object')
    
    digits = []
    for o in objects:
        name = o.find('name').text # reading bound box
        digits.append(name)
        
    return digits

def bounding_box(image):
    bpath=root_annots+"/"+str(image.split(".")[0]+".xml")
    tree = ET.parse(bpath)
    root = tree.getroot()
    objects = root.findall('object')
    
    for o in objects:
        bndbox = o.find('bndbox') # reading bound box
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)
        
        
    return (xmin,ymin,xmax,ymax)
    
plt.figure(figsize=(10,10))
bbox=[]
for i,image in enumerate(all_images):
    image = all_images[1]
    bbox=bounding_box(image) 
    digits = get_digits(image)
    im = cv.imread(os.path.join(root_images,image),cv.IMREAD_GRAYSCALE)
    im = cv.equalizeHist(im)
    
    im = im[bbox[1]:bbox[3],bbox[0]:bbox[2]]
    #cv.imshow("",im)
    for i in range(len(digits)):
      num = digits[i]
      
      ancho = int(im.shape[1]/len(digits))
      dig_im = im[0:im.shape[0],(i*ancho):(i*ancho)+ancho]
      dim = (28, 28)
      dig_im = cv.resize(dig_im, dim, interpolation = cv.INTER_AREA)
      #cv.imshow("",dig_im)
      cv.imwrite(carpeta_salida_entrenar+'/{}_{}.jpg'.format(num,uuid.uuid4().hex),dig_im)
print("Final")

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras.layers import MaxPool2D
from keras.layers import InputLayer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import compute_class_weight
from keras.callbacks import ModelCheckpoint
import cv2 as cv
import numpy as np
import random,os
from tensorflow.python.keras.models import load_model
from numpy.random import seed
from matplotlib import pyplot as plt
import math
import seaborn as sns

In [ ]:
def esPosible(entrada,filtro,num_filtros,padding,pool):
    actual = entrada
    for num in num_filtros:
        salida = ((actual - filtro[0]) / pool[0])
        if padding: salida = (actual / pool[0])
        actual = math.ceil(salida)
        if salida <= 0: 
            print("No se puede para ", entrada, filtro, num_filtros, padding)
            return 0
    return math.ceil(salida)

In [ ]:
def train(activation,padding,dropout,balanceo,optimizador,filtro,num_filtro,pool,capasPM):
    directorio = "./Resultados/"
    semilla=1
    valores = classes.values()
    claves = classes.keys()
    maximo = max(valores)
    pesos_clases = []
    for peso in valores:
        pesos_clases.append(maximo/peso)
    aux = []
    for clave in claves:
        aux.append(int(clave))
    claves = aux
    pesos_clases = dict(zip(claves,pesos_clases))
    seed(semilla)
    tf.random.set_seed(semilla)
    cadena = "ACTIVATION({})_PADDING({})_DROPOUT({})_BALANCEO({})_OPTIMIZADOR({})_FILTRO({})_FILTROS({})_POOL({})_CAPAS-PM({})".format(activation,padding,dropout,balanceo,optimizador,filtro,num_filtro,pool,capasPM)
    resultados = directorio + "LossFig"
    hechos = os.listdir(resultados)
    aux = cadena + "loss.png"
    if aux in hechos:
        print("Ya esta hecho")
        return 1
    if esPosible(28,filtro,num_filtro,padding,pool) > 0:
        salida = esPosible(28,filtro,num_filtro,padding,pool)
        salida = salida * salida * num_filtro[-1]
        #Esta es la parte de la CNN
        model = Sequential()
        model.add(BatchNormalization(input_shape=(28,28,1)))
        for num in num_filtro:
            if padding: model.add(Conv2D(num, filtro, padding='same', activation=activation))
            else: model.add(Conv2D(num, filtro, activation=activation))
            model.add(MaxPool2D(pool_size=pool))
            if dropout > 0: model.add(Dropout(dropout))
        model.add(Flatten())
        #Esta es la parte del PM
        for capa in range(capasPM):
            model.add(Dense(salida,activation=activation))
            salida = salida // 2
            if dropout > 0: model.add(Dropout(dropout))
        model.add(Dense(10, activation = "softmax"))
        model.compile(optimizer=optimizador,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
        model.summary(line_length=120)
        if not balanceo:
            historico = model.fit(x_train, y_train, 
                                  epochs=500, 
                                  validation_freq=1,
                                  validation_data=(x_test, y_test))
        else:
            historico = model.fit(x_train, y_train, 
                                  epochs=500, 
                                  validation_freq=1,
                                  validation_data=(x_test, y_test),
                                  class_weight=pesos_clases)
        ## plots de evolución de loss y accuracy
        
        from matplotlib import pyplot as plt
        plt.plot(historico.history['accuracy'])
        plt.plot(historico.history['val_accuracy'])
        plt.title('model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['train', 'val'], loc='upper left')
        plt.savefig(directorio + "AccFig/" + cadena + "accuracy.png")
        plt.show()
        plt.plot(historico.history['loss'])
        plt.plot(historico.history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'val'], loc='upper left')
        plt.show()
        plt.savefig(directorio + "LossFig/" + cadena + "loss.png")
        np.savetxt(directorio + "TrainLoss/" + cadena + 'historicoTrainLoss.txt',historico.history['loss'])
        np.savetxt(directorio + "ValLoss/" + cadena + 'historicoValLoss.txt',historico.history['val_loss'])
        np.savetxt(directorio + "TrainAcc/" + cadena + 'historicoTrainAcc.txt',historico.history['accuracy'])
        np.savetxt(directorio + "ValAcc/" + cadena + 'historicoValAcc.txt',historico.history['val_accuracy'])
        model.save(directorio + "Modelos/" + cadena + 'modelo.h5')
    return 0

In [ ]:
carpeta_salida_entrenar = "./Salida_para_entrenar"
all_images=os.listdir(carpeta_salida_entrenar)
random.shuffle(all_images)

_y = []
for image in all_images:
  _y.append(image.split("_")[0])


unique, counts = np.unique(_y, return_counts=True)
classes = dict(zip(unique, counts))
print(classes)

mean = np.mean(counts)
mean = int(mean + mean*0.2)

for i,count in zip(unique,counts):
  if count > mean:
    for j in range(count - mean):
      for image in all_images:
        if i == image.split("_")[0]:
          all_images.remove(image)
          break

train_images = all_images[:int(len(all_images)*0.8)]
test_images = all_images[int(len(all_images)*0.8):]
x_train = np.ndarray(shape=(len(train_images),28,28,1))
y_train = np.ndarray(shape=(len(train_images),))
x_test = np.ndarray(shape=(len(test_images),28,28,1))
y_test = np.ndarray(shape=(len(test_images),))
for i in range(len(train_images)):
  image = train_images[i]
  data = cv.imread(os.path.join(carpeta_salida_entrenar,image),cv.IMREAD_GRAYSCALE)
  x_train[i] = np.expand_dims(data, axis=-1)
  y_train[i] = image.split("_")[0]

for i in range(len(test_images)):
  image = test_images[i]
  data = cv.imread(os.path.join(carpeta_salida_entrenar,image),cv.IMREAD_GRAYSCALE)
  x_test[i] = np.expand_dims(data, axis=-1)
  y_test[i] = image.split("_")[0]

unique, counts = np.unique(y_train, return_counts=True)
print(dict(zip(unique, counts)))

In [ ]:
entrada=28
filtros = [(2,2), (3,3)]
num_filtros = [[32,16],[32],[32,16,8]]
padding = [True, False]
pools = [(2,2), (3,3)]
for fil in filtros:
    for num in num_filtros:
        for pad in padding:
            for pool in pools:
                esPosible(entrada,fil,num,pad,pool)
    

In [ ]:
import time
inicio = time.time()
activations = ["relu", "sigmoid", "tanh"]
padding = [True, False]
dropout = [0,0.25, 0.5, 0.75]
balanceo = [True, False]
optimizador = ["adam", "adamax", "nadam"]
filtros = [(2,2), (3,3)]
num_filtros = [[32,16],[32]]
pools = [(2,2), (3,3)]
capas_pms = [2,3] 
hechos = 0
for act in activations:
    for pad in padding:
        for drop in dropout:
            for bal in balanceo:
                for opt in optimizador:
                    for filtro in filtros:
                        for num in num_filtros:
                            for pool in pools:
                                for capa in capas_pms:
                                    hechos += train(act,pad,drop,bal,opt,filtro,num,pool,capa)
                                    print("Hechos: ",hechos)
fin = time.time()
print(fin-inicio)

In [ ]:
cadena=""
model = load_model(cadena)
evaluacion_cnn=model.evaluate(x_test, y_test)
#Obtencion de predicciones de CNN 
#predicciones en bruto:
raw_testPred_cnn = model.predict(x_test)
#predicciones de la clase:
class_testPred_cnn = np.argmax(raw_testPred_cnn, axis=1)
#Matriz de confusion y Classification report de CNN
cm_cnn=confusion_matrix(y_test, class_testPred_cnn)
print(cm_cnn)
sns.heatmap(cm_cnn,cmap="Greens",annot=True,xticklabels=classes.keys(),yticklabels=classes.keys())
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Matriz de confusión")
plt.show()
cm_normalized = np.round(cm_cnn/np.sum(cm_cnn,axis=1).reshape(-1,1),2)
print(cm_normalized)
sns.heatmap(cm_normalized,cmap="Greens",annot=True,xticklabels=classes.keys(),yticklabels=classes.keys())
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Matriz de confusión normalizada")
plt.show()

In [ ]:
!zip -r resultados.zip Resultados